In [2]:
import sys
sys.path.append('../../Share/')
sys.path.append('../../V5 Self Supervised Learning')

import pandas as pd
import numpy as np
from sklearn.utils import resample
import matplotlib.pyplot as plt
import baseline, config, self_supervised_v1

import warnings
warnings.filterwarnings('ignore')

def evaluate_model(model, data, labels):
    return model.evaluate(data, labels, verbose=0)[1]


def down_sample(X, y):
    X = np.array(X)
    y = np.array(y)

    # 클래스별 최소 개수
    unique_classes, counts = np.unique(y, return_counts=True)
    min_count = counts.min()

    X_balanced, y_balanced = [], []

    for cls in unique_classes:
        idx = np.where(y == cls)[0]  # 해당 클래스 인덱스
        down_idx = resample(idx, replace=False, n_samples=min_count, random_state=42)
        X_balanced.append(X[down_idx])
        y_balanced.append(y[down_idx])

    # 합치기
    X_balanced = np.concatenate(X_balanced, axis=0)
    y_balanced = np.concatenate(y_balanced, axis=0)

    return X_balanced, y_balanced



trainer_Minjeong = baseline.ModelTrainer(config, subject="Minjeong")
trainer_Carlson = baseline.ModelTrainer(config, subject="Carlson")
trainer_Harold = baseline.ModelTrainer(config, subject="Harold")
trainer_Hunmin = baseline.ModelTrainer(config, subject="Hunmin")
trainer_Brian = baseline.ModelTrainer(config, subject="Brian")
trainer_Xianyu = baseline.ModelTrainer(config, subject="Xianyu")

In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import meta

trainers_unseen = {
    "H": trainer_Hunmin,
    "C": trainer_Carlson,
    "B": trainer_Brian,
    "X": trainer_Xianyu,
    "M": trainer_Minjeong
}
#Same_Session_Test_Acc, Next_Session_Test_Acc = [], []
#Unseen_subject_acc_dict = {"H": [], "C": [], "B": [], "X": [], "M": []}

In [20]:
model = load_model('./model_K10_H2.h5')
model.compile(optimizer=Adam(learning_rate=0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-15-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-08-01-v1/E9AD0E7DCC2B/
0.5333333611488342
Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
0.5249999761581421
Returning K-th session data: Exp_2025-08-02-v1/E9AD0E7DCC2B/
0.5583333373069763
Returning K-th session data: Exp_2025-08-02-v2/E9AD0E7DCC2B/
0.6000000238418579
Returning K-th session data: Exp_2025-08-09-v1/E9AD0E7DCC2B/
0.6333333253860474
Returning K-th session data: Exp_2025-08-09-v2/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-08-10-v1/E9AD0E7DCC2B/
0.7166666388511658
Returning K-th session data: Exp_2025-08-10-v2/E9AD0E7DCC2B/
0.7666666507720947
Returning K-th session data: Exp_2025-08-11-v1/E9AD0E7DCC2B/
0.8083333373069763
Returning K-th session data: Exp_2025-08-11-v2/E9AD0E7DCC2B/
0.6166666746139526


In [21]:
np.min(Acc_lst), np.max(Acc_lst)

(np.float64(0.5249999761581421), np.float64(0.8083333373069763))

In [22]:
model = load_model('./model_K10_H2.h5')
model.compile(optimizer=Adam(learning_rate=0.05), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.5166666507720947
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.7250000238418579
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.8083333373069763
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.8416666388511658
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.875
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.875
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.875
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.9166666865348816
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.925000011920929
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.949999988079071


# Hunmin

In [23]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-15-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-08-01-v1/E9AD0E7DCC2B/
0.5333333611488342
Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
0.6166666746139526
Returning K-th session data: Exp_2025-08-02-v1/E9AD0E7DCC2B/
0.5666666626930237
Returning K-th session data: Exp_2025-08-02-v2/E9AD0E7DCC2B/
0.6166666746139526
Returning K-th session data: Exp_2025-08-09-v1/E9AD0E7DCC2B/
0.6666666865348816
Returning K-th session data: Exp_2025-08-09-v2/E9AD0E7DCC2B/
0.7333333492279053
Returning K-th session data: Exp_2025-08-10-v1/E9AD0E7DCC2B/
0.675000011920929
Returning K-th session data: Exp_2025-08-10-v2/E9AD0E7DCC2B/
0.6583333611488342
Returning K-th session data: Exp_2025-08-11-v1/E9AD0E7DCC2B/
0.8083333373069763
Returning K-th session data: Exp_2025-08-11-v2/E9AD0E7DCC2B/
0.625


In [24]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.8416666388511658
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.7166666388511658
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.8583333492279053
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.9166666865348816
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.925000011920929
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.9166666865348816
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.9166666865348816
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.925000011920929
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.9666666388511658


In [25]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Xianyu, len(config.Info_sub_X)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-07-23-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-24-v1/E9AD0E7DCC2B/
0.6583333611488342
Returning K-th session data: Exp_2025-06-24-v2/E9AD0E7DCC2B/
0.8166666626930237
Returning K-th session data: Exp_2025-06-26-v1/E9AD0E7DCC2B/
0.6666666865348816
Returning K-th session data: Exp_2025-06-26-v2/E9AD0E7DCC2B/
0.5583333373069763
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.6000000238418579
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.6333333253860474
Returning K-th session data: Exp_2025-06-30-v1/FEFFF6FFF5FF/
0.7333333492279053
Returning K-th session data: Exp_2025-06-30-v2/FEFFF6FFF5FF/
0.574999988079071
Returning K-th session data: Exp_2025-07-01-v1/E9AD0E7DCC2B/
0.574999988079071
Returning K-th session data: Exp_2025-07-01-v2/E9AD0E7DCC2B/
0.6333333253860474


In [26]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Carlson, len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-30-v1/E9AD0E7DCC2B/
0.5083333253860474
Returning K-th session data: Exp_2025-06-30-v2/E9AD0E7DCC2B/
0.6499999761581421
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.6083333492279053
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.6499999761581421
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.6833333373069763
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.6916666626930237
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.7583333253860474
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.6666666865348816
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.6416666507720947


In [27]:
model = load_model('./model_K10_H.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.7416666746139526
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.8833333253860474
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.9750000238418579
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.9166666865348816
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.925000011920929
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.9333333373069763
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.8666666746139526
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.9416666626930237
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.925000011920929
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.9166666865348816


# Xianyu

In [28]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.675000011920929
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.8500000238418579
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.8500000238418579
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.8916666507720947
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.9083333611488342
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.9083333611488342
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.8666666746139526
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.9416666626930237
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.875
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.9083333611488342


In [29]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Hunmin, len(config.Info_sub_H)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v3/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-05-27/E8331D05289A/
0.7166666388511658
Returning K-th session data: Exp_2025-06-18/E9AD0E7DCC2B/
0.699999988079071
Returning K-th session data: Exp_2025-06-20-v1/E9AD0E7DCC2B/
0.9083333611488342
Returning K-th session data: Exp_2025-06-20-v2/E9AD0E7DCC2B/
0.875
Returning K-th session data: Exp_2025-06-20-v3/E9AD0E7DCC2B/
0.925000011920929
Returning K-th session data: Exp_2025-06-20-v4/E9AD0E7DCC2B/
0.8416666388511658
Returning K-th session data: Exp_2025-06-20-v5/E9AD0E7DCC2B/
0.8416666388511658
Returning K-th session data: Exp_2025-06-20-v6/E9AD0E7DCC2B/
0.7666666507720947
Returning K-th session data: Exp_2025-06-20-v7/E9AD0E7DCC2B/
0.8500000238418579
Returning K-th session data: Exp_2025-06-20-v8/E9AD0E7DCC2B/
0.9083333611488342


In [30]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Carlson, len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-30-v1/E9AD0E7DCC2B/
0.4749999940395355
Returning K-th session data: Exp_2025-06-30-v2/E9AD0E7DCC2B/
0.5
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.5666666626930237
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.6083333492279053
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.7083333134651184
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.6833333373069763
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.7250000238418579
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.6666666865348816
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.7916666865348816


In [31]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.75
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.875
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.7833333611488342
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.8333333134651184
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.8416666388511658
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.8999999761581421
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.8999999761581421
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.9583333134651184
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.9416666626930237


In [32]:
model = load_model('./model_K10_X.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-15-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-08-01-v1/E9AD0E7DCC2B/
0.5916666388511658
Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
0.5666666626930237
Returning K-th session data: Exp_2025-08-02-v1/E9AD0E7DCC2B/
0.5916666388511658
Returning K-th session data: Exp_2025-08-02-v2/E9AD0E7DCC2B/
0.6166666746139526
Returning K-th session data: Exp_2025-08-09-v1/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-08-09-v2/E9AD0E7DCC2B/
0.7666666507720947
Returning K-th session data: Exp_2025-08-10-v1/E9AD0E7DCC2B/
0.6166666746139526
Returning K-th session data: Exp_2025-08-10-v2/E9AD0E7DCC2B/
0.6666666865348816
Returning K-th session data: Exp_2025-08-11-v1/E9AD0E7DCC2B/
0.7083333134651184
Returning K-th session data: Exp_2025-08-11-v2/E9AD0E7DCC2B/
0.6333333253860474


# Brian

In [33]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-15-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-08-01-v1/E9AD0E7DCC2B/
0.4749999940395355
Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
0.5249999761581421
Returning K-th session data: Exp_2025-08-02-v1/E9AD0E7DCC2B/
0.6000000238418579
Returning K-th session data: Exp_2025-08-02-v2/E9AD0E7DCC2B/
0.6666666865348816
Returning K-th session data: Exp_2025-08-09-v1/E9AD0E7DCC2B/
0.5833333134651184
Returning K-th session data: Exp_2025-08-09-v2/E9AD0E7DCC2B/
0.6583333611488342
Returning K-th session data: Exp_2025-08-10-v1/E9AD0E7DCC2B/
0.6583333611488342
Returning K-th session data: Exp_2025-08-10-v2/E9AD0E7DCC2B/
0.6583333611488342
Returning K-th session data: Exp_2025-08-11-v1/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-08-11-v2/E9AD0E7DCC2B/
0.6666666865348816


In [34]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Hunmin, len(config.Info_sub_H)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v3/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-05-27/E8331D05289A/
0.7250000238418579
Returning K-th session data: Exp_2025-06-18/E9AD0E7DCC2B/
0.675000011920929
Returning K-th session data: Exp_2025-06-20-v1/E9AD0E7DCC2B/
0.9416666626930237
Returning K-th session data: Exp_2025-06-20-v2/E9AD0E7DCC2B/
0.875
Returning K-th session data: Exp_2025-06-20-v3/E9AD0E7DCC2B/
0.9333333373069763
Returning K-th session data: Exp_2025-06-20-v4/E9AD0E7DCC2B/
0.8416666388511658
Returning K-th session data: Exp_2025-06-20-v5/E9AD0E7DCC2B/
0.8333333134651184
Returning K-th session data: Exp_2025-06-20-v6/E9AD0E7DCC2B/
0.7166666388511658
Returning K-th session data: Exp_2025-06-20-v7/E9AD0E7DCC2B/
0.800000011920929
Returning K-th session data: Exp_2025-06-20-v8/E9AD0E7DCC2B/
0.949999988079071


In [35]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.7916666865348816
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.9083333611488342
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.7250000238418579
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.7916666865348816
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.9166666865348816
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.9583333134651184
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.8416666388511658
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.9333333373069763
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.9083333611488342
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.9083333611488342


In [36]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Xianyu, len(config.Info_sub_X)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-07-23-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-24-v1/E9AD0E7DCC2B/
0.6499999761581421
Returning K-th session data: Exp_2025-06-24-v2/E9AD0E7DCC2B/
0.875
Returning K-th session data: Exp_2025-06-26-v1/E9AD0E7DCC2B/
0.6166666746139526
Returning K-th session data: Exp_2025-06-26-v2/E9AD0E7DCC2B/
0.5333333611488342
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.6166666746139526
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.675000011920929
Returning K-th session data: Exp_2025-06-30-v1/FEFFF6FFF5FF/
0.7583333253860474
Returning K-th session data: Exp_2025-06-30-v2/FEFFF6FFF5FF/
0.550000011920929
Returning K-th session data: Exp_2025-07-01-v1/E9AD0E7DCC2B/
0.5333333611488342
Returning K-th session data: Exp_2025-07-01-v2/E9AD0E7DCC2B/
0.6666666865348816


In [37]:
model = load_model('./model_K10_B.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Carlson, len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-30-v1/E9AD0E7DCC2B/
0.5083333253860474
Returning K-th session data: Exp_2025-06-30-v2/E9AD0E7DCC2B/
0.5916666388511658
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.699999988079071
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.7166666388511658
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.675000011920929
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.6666666865348816
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.7250000238418579
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.6916666626930237
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.7833333611488342


# Carlson

In [38]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.574999988079071
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.6833333373069763
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.8583333492279053
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.8666666746139526
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.9083333611488342
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.9166666865348816
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.8166666626930237
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.9416666626930237
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.9333333373069763
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.925000011920929


In [39]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Hunmin, len(config.Info_sub_H)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v3/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-05-27/E8331D05289A/
0.6583333611488342
Returning K-th session data: Exp_2025-06-18/E9AD0E7DCC2B/
0.675000011920929
Returning K-th session data: Exp_2025-06-20-v1/E9AD0E7DCC2B/
0.824999988079071
Returning K-th session data: Exp_2025-06-20-v2/E9AD0E7DCC2B/
0.8166666626930237
Returning K-th session data: Exp_2025-06-20-v3/E9AD0E7DCC2B/
0.8916666507720947
Returning K-th session data: Exp_2025-06-20-v4/E9AD0E7DCC2B/
0.75
Returning K-th session data: Exp_2025-06-20-v5/E9AD0E7DCC2B/
0.7916666865348816
Returning K-th session data: Exp_2025-06-20-v6/E9AD0E7DCC2B/
0.5583333373069763
Returning K-th session data: Exp_2025-06-20-v7/E9AD0E7DCC2B/
0.7333333492279053
Returning K-th session data: Exp_2025-06-20-v8/E9AD0E7DCC2B/
0.8666666746139526


In [40]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.5249999761581421
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.6499999761581421
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.800000011920929
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.7833333611488342
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.7333333492279053
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.800000011920929
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.8583333492279053
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.8916666507720947
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.9083333611488342
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.9333333373069763


In [41]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Xianyu, len(config.Info_sub_X)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-07-23-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-24-v1/E9AD0E7DCC2B/
0.5333333611488342
Returning K-th session data: Exp_2025-06-24-v2/E9AD0E7DCC2B/
0.6916666626930237
Returning K-th session data: Exp_2025-06-26-v1/E9AD0E7DCC2B/
0.6083333492279053
Returning K-th session data: Exp_2025-06-26-v2/E9AD0E7DCC2B/
0.5
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.6499999761581421
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.5666666626930237
Returning K-th session data: Exp_2025-06-30-v1/FEFFF6FFF5FF/
0.625
Returning K-th session data: Exp_2025-06-30-v2/FEFFF6FFF5FF/
0.5583333373069763
Returning K-th session data: Exp_2025-07-01-v1/E9AD0E7DCC2B/
0.6083333492279053
Returning K-th session data: Exp_2025-07-01-v2/E9AD0E7DCC2B/
0.6583333611488342


In [42]:
model = load_model('./model_K10_C.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Minjeong, len(config.Info_sub_M)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-15-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-08-01-v1/E9AD0E7DCC2B/
0.5249999761581421
Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
0.625
Returning K-th session data: Exp_2025-08-02-v1/E9AD0E7DCC2B/
0.6083333492279053
Returning K-th session data: Exp_2025-08-02-v2/E9AD0E7DCC2B/
0.6416666507720947
Returning K-th session data: Exp_2025-08-09-v1/E9AD0E7DCC2B/
0.6833333373069763
Returning K-th session data: Exp_2025-08-09-v2/E9AD0E7DCC2B/
0.6833333373069763
Returning K-th session data: Exp_2025-08-10-v1/E9AD0E7DCC2B/
0.7083333134651184
Returning K-th session data: Exp_2025-08-10-v2/E9AD0E7DCC2B/
0.6916666626930237
Returning K-th session data: Exp_2025-08-11-v1/E9AD0E7DCC2B/
0.6833333373069763
Returning K-th session data: Exp_2025-08-11-v2/E9AD0E7DCC2B/
0.6666666865348816


# Minjeong

In [43]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Xianyu, len(config.Info_sub_X)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-07-23-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-24-v1/E9AD0E7DCC2B/
0.5666666626930237
Returning K-th session data: Exp_2025-06-24-v2/E9AD0E7DCC2B/
0.7166666388511658
Returning K-th session data: Exp_2025-06-26-v1/E9AD0E7DCC2B/
0.7749999761581421
Returning K-th session data: Exp_2025-06-26-v2/E9AD0E7DCC2B/
0.550000011920929
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.7083333134651184
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.6916666626930237
Returning K-th session data: Exp_2025-06-30-v1/FEFFF6FFF5FF/
0.625
Returning K-th session data: Exp_2025-06-30-v2/FEFFF6FFF5FF/
0.6499999761581421
Returning K-th session data: Exp_2025-07-01-v1/E9AD0E7DCC2B/
0.7416666746139526
Returning K-th session data: Exp_2025-07-01-v2/E9AD0E7DCC2B/
0.6666666865348816


In [44]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Hunmin, len(config.Info_sub_H)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v3/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-05-27/E8331D05289A/
0.7666666507720947
Returning K-th session data: Exp_2025-06-18/E9AD0E7DCC2B/
0.6916666626930237
Returning K-th session data: Exp_2025-06-20-v1/E9AD0E7DCC2B/
0.8666666746139526
Returning K-th session data: Exp_2025-06-20-v2/E9AD0E7DCC2B/
0.8999999761581421
Returning K-th session data: Exp_2025-06-20-v3/E9AD0E7DCC2B/
0.9166666865348816
Returning K-th session data: Exp_2025-06-20-v4/E9AD0E7DCC2B/
0.7583333253860474
Returning K-th session data: Exp_2025-06-20-v5/E9AD0E7DCC2B/
0.800000011920929
Returning K-th session data: Exp_2025-06-20-v6/E9AD0E7DCC2B/
0.699999988079071
Returning K-th session data: Exp_2025-06-20-v7/E9AD0E7DCC2B/
0.8583333492279053
Returning K-th session data: Exp_2025-06-20-v8/E9AD0E7DCC2B/
0.875


In [45]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Harold, len(config.Info_sub_H2)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.7333333492279053
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.7583333253860474
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.8833333253860474
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.8916666507720947
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.9083333611488342
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.9333333373069763
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.8583333492279053
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.925000011920929
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.8916666507720947
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.9166666865348816


In [46]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Brian, len(config.Info_sub_B)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-27-v1/E9AD0E7DCC2B/
0.5916666388511658
Returning K-th session data: Exp_2025-06-27-v2/E9AD0E7DCC2B/
0.5833333134651184
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.75
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.8416666388511658
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.800000011920929
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.8583333492279053
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.8666666746139526
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.8833333253860474
Returning K-th session data: Exp_2025-07-17-v1/E9AD0E7DCC2B/
0.9750000238418579
Returning K-th session data: Exp_2025-07-17-v2/E9AD0E7DCC2B/
0.9583333134651184


In [47]:
model = load_model('./model_K10_M.h5')
model.compile(optimizer=Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer_sub, final_session = trainer_Carlson, len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)
X_TEST, y_TEST = down_sample(X_TEST, y_TEST)
Acc_lst = []

for k in range(10):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=1                          # Reptile meta step
    )

    acc = Meta.train(X_train, y_train, X_TEST, y_TEST, meta.get_data_Meta, N_way=6, K_shot=20)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-30-v1/E9AD0E7DCC2B/
0.4416666626930237
Returning K-th session data: Exp_2025-06-30-v2/E9AD0E7DCC2B/
0.550000011920929
Returning K-th session data: Exp_2025-07-09-v1/E9AD0E7DCC2B/
0.6333333253860474
Returning K-th session data: Exp_2025-07-09-v2/E9AD0E7DCC2B/
0.6916666626930237
Returning K-th session data: Exp_2025-07-10-v1/E9AD0E7DCC2B/
0.75
Returning K-th session data: Exp_2025-07-10-v2/E9AD0E7DCC2B/
0.6833333373069763
Returning K-th session data: Exp_2025-07-11-v1/E9AD0E7DCC2B/
0.7416666746139526
Returning K-th session data: Exp_2025-07-11-v2/E9AD0E7DCC2B/
0.6833333373069763
Returning K-th session data: Exp_2025-07-16-v1/E9AD0E7DCC2B/
0.7250000238418579
Returning K-th session data: Exp_2025-07-16-v2/E9AD0E7DCC2B/
0.6333333253860474
